In [47]:
### CLEAN - NEW Weather Scraper Book 

## Load Libraries
import requests
import json
import pandas as pd
import numpy as np
import datetime
import re
from time import sleep
import time

from tqdm import tqdm

path = '../data/stats/ncaa/no_boxscore_data_v1.csv'

## Load Data
df = pd.read_csv(path)
# remove rows with no lat long data
df = df.dropna(subset=['latitude', 'longitude'])

API_KEY = '0663fa202f1f8ea3d4a9462600bd449d'
# API_KEY =  '29e69059f84d17c13de92f5bb94f4fb8'

In [48]:
df.info()

df.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10975 entries, 1 to 12708
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              10975 non-null  object 
 1   away_team         10975 non-null  object 
 2   home_team         10975 non-null  object 
 3   game_info         1467 non-null   object 
 4   away_team_runs    10975 non-null  float64
 5   away_team_hits    10975 non-null  float64
 6   away_team_errors  10975 non-null  float64
 7   home_team_runs    10975 non-null  float64
 8   home_team_hits    10975 non-null  float64
 9   home_team_errors  10975 non-null  float64
 10  campus_site       10975 non-null  object 
 11  year              10975 non-null  int64  
 12  team              10833 non-null  object 
 13  latitude          10975 non-null  float64
 14  longitude         10975 non-null  float64
 15  elevation         9508 non-null   float64
 16  place             1467 non-null   object

,date,away_team,home_team,game_info,away_team_runs,away_team_hits,away_team_errors,home_team_runs,home_team_hits,home_team_errors,campus_site,year,team,latitude,longitude,elevation,place,n_venue
1,2016-02-19,Seton Hall,Missouri,AT FORT MYERS FL,0.0,4.0,1.0,7.0,12.0,0.0,unknown,2016,Missouri,26.634254,-81.867859,NaN,City of Palms Park,City of Palms Park
2,2016-02-19,Seton Hall,Missouri,AT FORT MYERS FL,0.0,4.0,1.0,7.0,12.0,0.0,unknown,2016,Missouri,26.551258,-81.761830,NaN,JetBlue Park,JetBlue Park
3,2016-02-19,Seton Hall,Missouri,AT FORT MYERS FL,0.0,4.0,1.0,7.0,12.0,0.0,unknown,2016,Missouri,26.537668,-81.842247,NaN,Hammond Stadium,Hammond Stadium
4,2016-02-19,SIU Edwardsville,Louisville,NaN,4.0,8.0,2.0,21.0,17.0,1.0,Louisville,2016,Louisville,38.205516,-85.762865,140.286621,NaN,NaN
5,2016-02-19,Central Michigan,Arkansas,NaN,1.0,2.0,3.0,6.0,8.0,0.0,Arkansas,2016,Arkansas,36.049888,-94.182241,374.505188,NaN,NaN


In [49]:
# ## Take a sample of 10 to test the function
# df_sample = df.sample(5)
# df = df_sample

# set starting index
starting_index = 0



In [50]:
## Get the weather data
# empty list to hold the results
results = []

from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

@retry(stop=stop_after_attempt(5), 
       wait=wait_exponential(multiplier=1, min=2, max=10), 
       retry=retry_if_exception_type(requests.HTTPError))
def get_weather_data(url):
    response = requests.get(url)
    if response.status_code == 429:
        retry_after = int(response.headers.get('Retry-After', 0))
        print(f"Rate limited. Waiting for {retry_after} seconds.")
        time.sleep(retry_after)
        response = requests.get(url)
    response.raise_for_status()
    return response.json()

## Get the weather data for all of the games
# Loop through the rows in the df
for index, row in tqdm(df.iterrows()):
    if index > starting_index:
        time.sleep(3)
        # Get the latitude and longitude for the current row
        latitude = row['latitude']
        longitude = row['longitude']
        # Get the Unix timestamp for the current row
        # unix_timestamp = row['unix_timestamp']
        # get the simple date, make sure it is a string
        date = row['date']
        # Create the API request URL
        url = f'https://api.openweathermap.org/data/3.0/onecall/day_summary?lat={latitude}&lon={longitude}&date={date}&appid={API_KEY}&units=imperial'
        # Try to make the API request
        try:
            weather_data = get_weather_data(url)
            results.append({'index': index, 'weather_data': weather_data})
        # If the request was unsuccessful, print the response code
        except requests.HTTPError as e:
            print(f"Could not get data for row {index}. HTTPError: {e}")
            results.append({'index': index, 'weather_data': 'failed'})
            df_results = pd.DataFrame(results)
            df_results.to_csv(f'../data/stats/ncaa/WEATHER_partial__V1_{index+1}.csv', index=False)
            print(f"Data saved at index {index+1}")
            continue



# add the results to a dataframe
df_results = pd.DataFrame(results)



# merge the results with the original df
df = df.merge(df_results, left_index=True, right_on='index')

# save the results to a csv
df.to_csv('../data/stats/ncaa/no_boxscore_with_weather_v1.csv', index=False)



1354it [1:35:56,  4.42s/it]